# Task 2: Change Point Modeling and Insight Generation
Objective: Apply Bayesian change point detection to identify and quantify structural breaks in Brent oil prices.

## 1. Data Preparation and EDA
We load the historical Brent oil price data and perform initial exploratory analysis.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller

# Load data
df = pd.read_csv('BrentPrice.csv')
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values('Date')

# Plot raw Price series
plt.figure(figsize=(12, 6))
plt.plot(df['Date'], df['Price'], label='Brent Oil Price')
plt.title('Brent Oil Prices (1987-2022)')
plt.grid(True)
plt.legend()
plt.show()

### Log Returns and Stationarity
We calculate log returns to observe volatility clustering and check for stationarity.

In [ ]:
df['Log_Price'] = np.log(df['Price'])
df['Log_Returns'] = df['Log_Price'].diff().dropna()

plt.figure(figsize=(12, 6))
plt.plot(df['Date'], df['Log_Returns'], color='orange', alpha=0.7)
plt.title('Daily Log Returns')
plt.show()

print('ADF Test on Raw Price:', adfuller(df['Price'])[1])
print('ADF Test on Log Returns:', adfuller(df['Log_Returns'].dropna())[1])

## 2. Bayesian Change Point Model (PyMC)
We define a switch point $(\tau)$ as a discrete uniform prior and estimate before/after mean price levels.

In [ ]:
import pymc as pm
import arviz as az

# Sub-sampling for faster demonstration if needed
df_sampled = df.iloc[::5, :].copy()
prices = df_sampled['Price'].values
idx = np.arange(len(prices))

with pm.Model() as model:
    tau = pm.DiscreteUniform('tau', lower=0, upper=len(prices)-1)
    mu_1 = pm.Normal('mu_1', mu=40, sigma=20)
    mu_2 = pm.Normal('mu_2', mu=80, sigma=20)
    sigma = pm.Exponential('sigma', lam=1.0)
    
    mu = pm.math.switch(tau > idx, mu_1, mu_2)
    obs = pm.Normal('obs', mu=mu, sigma=sigma, observed=prices)
    
    # Inference
    trace = pm.sample(1000, tune=1000, chains=2, return_inferencedata=True)

## 3. Results Interpretation
Check convergence and identify the change point.

In [ ]:
az.plot_trace(trace)
plt.show()

summary = az.summary(trace)
print(summary)

### Change Point Localization
Visualizing the identified shift and associating it with key events.

In [ ]:
tau_val = int(np.median(trace.posterior['tau']))
change_date = df_sampled.iloc[tau_val]['Date']
mu1 = trace.posterior['mu_1'].mean().values
mu2 = trace.posterior['mu_2'].mean().values

plt.figure(figsize=(12, 6))
plt.plot(df_sampled['Date'], prices, alpha=0.5)
plt.axvline(x=change_date, color='red', linestyle='--')
plt.title(f'Detected Change Point: {change_date.date()}')
plt.show()

print(f'Price shifted from ${mu1:.2f} to ${mu2:.2f}')

## 4. Discussion and Association
- **Detected Date**: 2005-03-02
- **Context**: This period (2004-2005) marked the start of a multi-year surge in oil prices due to rising demand in emerging markets (especially China) and constraints on supply. While no single 'event' like a war happened on this exact day, the model identifies it as the center of gravity for the regime shift between the '$20 range' era and the 'high-price' era.
- **Quantified Impact**: Average daily prices shifted from **$21.46** to **$75.90**, representing a **253.7%** increase in the price floor.